From the following from ORIG, we have to formalize a GRL problem.

And reading the two papers: `../papers/Key-Players-GNN-19.pdf` and `../papers/DarkWeb-Pred.pdf`

What all is useful here:
- orig_members: member_id (763)
- orig_inline_notifications: notify_from_id, notify_to_id (47066)
- orig_message_topic_user_map: map_user_id, map_topic_id (6025)
- orig_message_topics: mt_starter_id, mt_to_member_id, mt_id (3101)
- orig_reputation_index: member_id (141550)

NOTE: Make sure to adjust some inconsistencies in members before trying out GRL.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import requests 
import io
import seaborn as sns

Graph Formulation:

**Nodes**: Members (total 763)

**Node Features**: (the node label for classification)
- Reputation Value 

**Edges**: (both can be treated as directed edges)
- Notifications (total 47066)
- Private Messages (total 3101)

In [23]:
PATH = '../data/'
files = os.listdir(PATH)
dfs = {f[:-4] : pd.read_csv(PATH + f) 
           for f in files if f[-3:] == 'csv'
}

Now, we need to fit this problem into the Vanilla GCN implementation by Kipf!

In [26]:
# first, make Python lists of all the relevant data
members = sorted(list(dfs['orig_members'].member_id))

In [31]:
dfs['orig_members'].shape

(763, 74)

In [35]:
reputation = [[m, sum(dfs['orig_reputation_index'].member_id == m)]
              for m in members]

In [45]:
# maybe using 17.0 as the cutoff
just_reps = [reputation[z][1] for z in range(reputation.__len__())]
med_reps  = np.median(just_reps)
med_reps

17.0

In [51]:
notifs_raw = [[dfs['orig_inline_notifications'].notify_from_id[z], 
               dfs['orig_inline_notifications'].notify_to_id[z]]
              for z in range(dfs['orig_inline_notifications'].shape[0])]

In [55]:
notifs = [n for n in notifs_raw if (n[0] in members and n[1] in members)]

In [59]:
messages_raw = [[dfs['orig_message_topics'].mt_starter_id[z], 
               dfs['orig_message_topics'].mt_to_member_id[z]]
              for z in range(dfs['orig_message_topics'].shape[0])]

In [61]:
messages = [n for n in messages_raw if (n[0] in members and n[1] in members)]

In [62]:
len(messages_raw), len(messages)

(3101, 2859)